In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk...."

In [ ]:
pip install pymupdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader

# load pdf

In [ ]:
loader = PyMuPDFLoader("fewshot_learning.pdf")
docs = loader.load()

In [ ]:
docs[0].metadata

{'producer': 'pdfTeX-1.40.17',
 'creator': 'LaTeX with hyperref package',
 'creationdate': '2020-07-24T00:04:08+00:00',
 'source': 'fewshot_learning.pdf',
 'file_path': 'fewshot_learning.pdf',
 'total_pages': 75,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '2020-07-24T00:04:08+00:00',
 'trapped': '',
 'modDate': 'D:20200724000408Z',
 'creationDate': 'D:20200724000408Z',
 'page': 0}

# text splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [ ]:
chunks[0].page_content

'Language Models are Few-Shot Learners\nTom B. Brown∗\nBenjamin Mann∗\nNick Ryder∗\nMelanie Subbiah∗\nJared Kaplan†\nPrafulla Dhariwal\nArvind Neelakantan\nPranav Shyam\nGirish Sastry\nAmanda Askell\nSandhini Agarwal\nAriel Herbert-Voss\nGretchen Krueger\nTom Henighan\nRewon Child\nAditya Ramesh\nDaniel M. Ziegler\nJeffrey Wu\nClemens Winter\nChristopher Hesse\nMark Chen\nEric Sigler\nMateusz Litwin\nScott Gray\nBenjamin Chess\nJack Clark\nChristopher Berner\nSam McCandlish\nAlec Radford\nIlya Sutskever\nDario Amodei\nOpenAI'

# embeddings and vector store

In [ ]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embedding_model)

# vector store as retriever

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",     # or "similarity" or mmr
    search_kwargs={"k": 5}
)

docs = retriever.invoke("Who wrote the paper?")
print(docs)

[Document(id='d0b16b94-339b-4f31-b7e6-bd596910fb63', metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-07-24T00:04:08+00:00', 'source': 'fewshot_learning.pdf', 'file_path': 'fewshot_learning.pdf', 'total_pages': 75, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2020-07-24T00:04:08+00:00', 'trapped': '', 'modDate': 'D:20200724000408Z', 'creationDate': 'D:20200724000408Z', 'page': 41}, page_content='broader impacts analysis, and advised Gretchen, Amanda, Girish, Sandhini, and Ariel on their work.\nDario Amodei, Alec Radford, Tom Brown, Sam McCandlish, Nick Ryder, Jared Kaplan, Sandhini Agarwal,\nAmanda Askell, Girish Sastry, and Jack Clark wrote the paper.\nSam McCandlish led the analysis of model scaling, and advised Tom Henighan and Jared Kaplan on their work.\nAlec Radford advised the project from an NLP perspective, suggested tasks, put the results in context, and demonstrated'), Docume

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(),
    llm=llm
)

docs = retriever.invoke("Compare zeroshot, one shott and few shot learning")
print(docs)

[Document(id='4df70a45-3f9e-4610-8fec-1132222b24de', metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-07-24T00:04:08+00:00', 'source': 'fewshot_learning.pdf', 'file_path': 'fewshot_learning.pdf', 'total_pages': 75, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2020-07-24T00:04:08+00:00', 'trapped': '', 'modDate': 'D:20200724000408Z', 'creationDate': 'D:20200724000408Z', 'page': 6}, page_content='zero-shot, one-shot and few-shot, with the aim of comparing them not as competing alternatives, but as different\nproblem settings which offer a varying trade-off between performance on speciﬁc benchmarks and sample efﬁciency.\nWe especially highlight the few-shot results as many of them are only slightly behind state-of-the-art ﬁne-tuned models.\nUltimately, however, one-shot, or even sometimes zero-shot, seem like the fairest comparisons to human performance,'), Document(id='2a43aa4a-c5e6-43c6-

# Augmentation

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.documents import Document

# Step to get docs from retriever and join as a string
def get_context(question: str):
    docs = retriever.invoke(question)
    return "\n\n".join(doc.page_content for doc in docs)

parallel_chain = RunnableMap({
    "question": RunnablePassthrough(),       # keep original question
    "context": get_context         # fetch & format docs
})

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Describe the paper in detail?')

'The paper discusses the training and evaluation of GPT-3, organized into several sections. \n\n1. **Introduction**: It sets the stage for the importance of informal conversation in business relationships and the need to understand cultural sensitivities.\n\n2. **Approach**: This section details the methods used for training GPT-3 and evaluating its performance.\n\n3. **Results**: It presents findings on various tasks in zero-, one-, and few-shot settings.\n\n4. **Data Contamination**: This section addresses concerns regarding train-test overlap.\n\n5. **Limitations**: It discusses the weaknesses of GPT-3 in text synthesis and several NLP tasks, despite improvements over its predecessor, GPT-2.\n\n6. **Broader Impacts**: This section explores the wider implications of the technology.\n\n7. **Related Work**: It reviews existing literature relevant to the study.\n\n8. **Conclusion**: The paper wraps up the findings and discussions.\n\nAdditionally, the authors include specific figures il

In [ ]:
main_chain.invoke('Describe the broader impacts in brief?')

'The broader impacts of language models like GPT-3 include beneficial applications such as code and writing auto-completion, grammar assistance, game narrative generation, improving search engine responses, and answering questions. However, there are significant concerns regarding the potential for deliberate misuse of these models, as well as issues of bias, fairness, and representation within the models. Additionally, energy efficiency is a concern due to the large amounts of computation required for training these models.'

In [ ]:
from IPython.display import Markdown, display
response = main_chain.invoke('compare zero shot, one shot, few shot learning in detail')
display(Markdown(response))

Zero-shot, one-shot, and few-shot learning are different problem settings that offer varying trade-offs between performance on specific benchmarks and sample efficiency.

- **Zero-shot learning** involves the model performing a task without any prior examples or demonstrations. The model relies solely on its pre-trained knowledge and a natural language description of the task.

- **One-shot learning** allows for only one demonstration of the task, in addition to the natural language description. This setting is significant because it closely resembles how some tasks are communicated to humans.

- **Few-shot learning** permits the model to see a few dozen examples before performing the task. This approach often yields results that are only slightly behind state-of-the-art fine-tuned models.

The comparison highlights that while few-shot learning can achieve high performance, zero-shot and one-shot learning are often seen as fairer comparisons to human performance, as they require less reliance on examples. Each method captures different aspects of learning and adaptation, with few-shot being more sample-efficient than traditional fine-tuning methods.

In [ ]:
#with multiquery retriever
from IPython.display import Markdown, display
response = main_chain.invoke('compare zero shot, one shot, few shot learning in detail')
display(Markdown(response))

Zero-shot, one-shot, and few-shot learning are different problem settings that offer varying trade-offs between performance on specific benchmarks and sample efficiency.

- **Zero-shot learning** involves the model performing a task without any prior examples or demonstrations. The model relies solely on its pre-trained knowledge and a natural language description of the task.

- **One-shot learning** allows the model to see only one demonstration of the task, in addition to the natural language description. This setting is significant because it closely resembles how some tasks are communicated to humans, making it a relevant comparison for human performance.

- **Few-shot learning** permits the model to be presented with a few dozen examples of the task, along with the natural language description. This approach often yields results that are only slightly behind state-of-the-art fine-tuned models, highlighting its effectiveness.

Overall, while fine-tuning is the traditional method for training models, zero-shot, one-shot, and few-shot learning require only forward passes at test time, making them more sample efficient. Each method has its own strengths and is suited for different scenarios, with few-shot learning often providing a good balance between performance and efficiency.